# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 48 new papers today
          22 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/21 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.15015


extracting tarball to tmp_2209.15015...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.15019


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.15015/natnotes.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2209.15581...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.15590


extracting tarball to tmp_2209.15590...

 done.


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.15590/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.15110-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.15110) | **High Altitude characterization of the Hunga Pressure Wave with Cosmic  Rays by the HAWC Observatory**  |
|| <mark>R. Alfaro</mark>, et al. -- incl., <mark>R. Diaz Hernandez</mark>, <mark>S. Hernandez</mark>, <mark>A. Lara</mark>, <mark>J.T. Linnemann</mark> |
|*Appeared on*| *2022-10-03*|
|*Comments*| *Contact about this analysis: A. Sandoval (asandoval@fisica.unam.mx), A. Lara (alara@igeofisica.unam.mx) & H. Le\'on Vargas (hleonvar@fisica.unam.mx)*|
|**Abstract**| High-energy cosmic rays that hit the Earth can be used to study large-scale atmospheric perturbations. After a first interaction in the upper parts of the atmosphere, cosmic rays produce a shower of particles that sample the atmosphere down to the detector level. The HAWC (High-Altitude Water Cherenkov) cosmic-ray observatory in Central Mexico at 4,100 m elevation detects air shower particles continuously with 300 water Cherenkov detectors with an active area of 12,500 m$^{2}$. On January 15th, 2022, HAWC detected the passage of the pressure wave created by the explosion of the Hunga volcano in the Tonga islands, 9,000 km away, as an anomaly in the measured rate of shower particles. The HAWC measurements are used to characterize the shape of four pressure wave passages, determine the propagation speed of each one, and correlate the variations of the shower particle rates with the barometric pressure changes, extracting a barometric parameter. The profile of the shower particle rate and atmospheric pressure variations for the first transit of the pressure wave at HAWC is compared to the pressure measurements at Tonga island, near the volcanic explosion. This work opens the possibility of using large particle cosmic-ray air shower detectors to trace large atmospheric transient waves. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.15403-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.15403) | **Atmospheric retrievals for LIFE and other future space missions: the  importance of mitigating systematic effects**  |
|| Eleonora Alei, et al. -- incl., <mark>Paul Mollière</mark> |
|*Appeared on*| *2022-10-03*|
|*Comments*| *24 pages, 12 figures. Proceedings SPIE Volume 12180, Space Telescopes and Instrumentation 2022: Optical, Infrared, and Millimeter Wave; 121803L (2022)*|
|**Abstract**| Atmospheric retrieval studies are essential to determine the science requirements for future generation missions, such as the Large Interferometer for Exoplanets (LIFE). The use of heterogeneous absorption cross-sections might be the cause of systematic effects in retrievals, which could bias a correct characterization of the atmosphere. In this contribution we quantified the impact of differences in line list provenance, broadening coefficients, and line wing cut-offs in the retrieval of an Earth twin exoplanet orbiting a Sun-like star at 10 pc from the observer, as it would be observed with LIFE. We ran four different retrievals on the same input spectrum, by varying the opacity tables that the Bayesian retrieval framework was allowed to use. We found that the systematics introduced by the opacity tables could bias the correct estimation of the atmospheric pressure at the surface level, as well as an accurate retrieval of the abundance of some species in the atmosphere (such as CO$_2$ and N$_2$O). We argue that differences in the line wing cut-off might be the major source of errors. We highlight the need for more laboratory and modeling efforts, as well as inter-model comparisons of the main radiative transfer models and Bayesian retrieval frameworks. This is especially relevant in the context of LIFE and future generation missions, to identify issues and critical points for the community to jointly work together to prepare for the analysis of the upcoming observations. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.15417-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.15417) | **Multiple gas acquisition events in galaxies with dual misaligned gas  disks**  |
|| Xiao Cao, et al. -- incl., <mark>Song-Lin Li</mark> |
|*Appeared on*| *2022-10-03*|
|*Comments*| *Published in Nature Astronomy on 29 September 2022; 23 pages, 6 figures*|
|**Abstract**| Frequent accretion of external cold gas is thought to play an important role in galaxy assembly. However, almost all known kinematically misaligned galaxies identify only one gas disk that is misaligned with the stellar disk, implying a single gas acquisition event. Here we report a new configuration in two galaxies where both contain two gas disks misaligned with each other and also with the stellar disk. Such systems are not expected to be stable or long-lasting, challenging the traditional picture of gas accretion of galaxies and their angular momentum build-up. The differences in kinematic position angles are larger than 120{\deg} between the two gas disks, and 40{\deg} between each gas disk and the stellar component. The star formation activity is enhanced at the interface of the two gas disks compared with the other regions within the same galaxy. Such systems illustrate that low-redshift galaxies can still experience multiple gas acquisition events, and provide a new view into the origins of galactic gas. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.15467-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.15467) | **The 700 ks Chandra Spiderweb Field II: Evidence for inverse-Compton and  thermal diffuse emission in the Spiderweb galaxy**  |
|| P. Tozzi, et al. -- incl., <mark>A. Liu</mark> |
|*Appeared on*| *2022-10-03*|
|*Comments*| *19 pages, 16 figures, accepted in Astronomy & Astrophysics*|
|**Abstract**| We present the X-ray imaging and spectral analysis of the diffuse emission around the Spiderweb galaxy at z=2.16 and of its nuclear emission, based on a deep (700 ks) Chandra observation. We characterize the nuclear emission and computed the contamination in the surrounding regions due to the wings of the instrument PSF. Then, we quantified the extended emission within 12". We find that the Spiderweb galaxy hosts a mildly absorbed quasar, with modest yet significant variability on a timescale of ~1 year. We find that the emission in the jet regions is well described by a power law with Gamma~2-2.5, and it is consistent with IC upscattering of the CMB photons by the relativistic electrons. We also find a roughly symmetric, diffuse emission within a radius of ~100 kpc. This emission is consistent with thermal bremsstrahlung from a hot ICM with a temperature of kT=2.0_{-0.4}^{+0.7} keV, and a metallicity of Z<1.6Z_sun. The average electron density within 100 kpc is n_e=(1.51+-0.24+-0.14)E-2 cm^{-3}, corresponding to an upper limit for the total ICM mass of <=(1.76+-0.30+-0.17)E+12 M_sun (where error bars are 1 sigma statistical and systematic, respectively). If we apply hydrostatic equilibrium to the ICM, we measure a total gravitational mass M(<100 kpc)=(1.5^{+0.5}_{-0.3})E+13 M_sun and, extrapolating at larger radii, we estimate a total mass M_{500}=(3.2^{+1.1}_{-0.6})E+13 M_sun within a radius of r_{500}=(220+-30) kpc. We conclude that the Spiderweb protocluster shows significant diffuse emission within a radius of 12 arcsec, whose major contribution is provided by IC scattering associated with the radio jets. Outside the jet regions, we also identified thermal emission within a radius of ~100 kpc, revealing the presence of hot, diffuse baryons that may represent the embryonic virialized halo of the forming cluster. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.15548-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.15548) | **The Roasting Marshmallows Program with IGRINS on Gemini South I:  Composition and Climate of the Ultra Hot Jupiter WASP-18 b**  |
|| Matteo Brogi, et al. -- incl., <mark>Michael R. Line</mark>, <mark>Peter Smith</mark>, <mark>Joseph A. Zalesky</mark>, <mark>Lisa Dang</mark> |
|*Appeared on*| *2022-10-03*|
|*Comments*| *27 pages, 18 figures, submitted to AAS Journals. Community feedback welcome*|
|**Abstract**| We present high-resolution dayside thermal emission observations of the exoplanet WASP-18b using IGRINS on Gemini South. We remove stellar and telluric signatures using standard algorithms, and we extract the planet signal via cross correlation with model spectra. We detect the atmosphere of WASP-18b at a signal-to-noise ratio (SNR) of 5.9 using a full chemistry model, measure H2O (SNR=3.3), CO (SNR=4.0), and OH (SNR=4.8) individually, and confirm previous claims of a thermal inversion layer. The three species are confidently detected (>4$\sigma$) with a Bayesian inference framework, which we also use to retrieve abundance, temperature, and velocity information. For this ultra-hot Jupiter (UHJ), thermal dissociation processes likely play an important role. Retrieving abundances constant with altitude and allowing the temperature-pressure profile to freely adjust results in a moderately super-stellar carbon to oxygen ratio (C/O=0.75^{+0.14}_{-0.17}) and metallicity ([M/H]=1.03^{+0.65}_{-1.01}). Accounting for undetectable oxygen produced by thermal dissociation leads to C/O=0.45^{+0.08}_{-0.10} and [M/H]=1.17^{+0.66}_{-1.01}. A retrieval that assumes radiative-convective-thermochemical-equilibrium and naturally accounts for thermal dissociation constrains C/O<0.34 (2$\sigma$) and [M/H]=0.48^{+0.33}_{-0.29}, in line with the chemistry of the parent star. Looking at the velocity information, we see a tantalising signature of different Doppler shifts at the level of a few km/s for different molecules, which might probe dynamics as a function of altitude and location on the planet disk. Our results demonstrate that ground-based, high-resolution spectroscopy at infrared wavelengths can provide meaningful constraints on the compositions and climate of highly irradiated planets. This work also elucidates potential pitfalls with commonly employed retrieval assumptions when applied to UHJ spectra. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.15581-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.15581) | **Neutrino propagation in the Earth and emerging charged leptons with  $\texttt{nuPyProp}$**  |
|| Diksha Garg, et al. -- incl., <mark>Tobias Heibges</mark> |
|*Appeared on*| *2022-10-03*|
|*Comments*| *42 pages, 21 figures, code available at this https URL*|
|**Abstract**| Ultra-high-energy neutrinos serve as messengers of some of the highest energy astrophysical environments. Given that neutrinos are neutral and only interact via weak interactions, neutrinos can emerge from sources, traverse astronomical distances, and point back to their origins. Their weak interactions require large target volumes for neutrino detection. Using the Earth as a neutrino converter, terrestrial, sub-orbital, and satellite-based instruments are able to detect signals of neutrino-induced extensive air showers. In this paper, we describe the software code $\texttt{nuPyProp}$ that simulates tau neutrino and muon neutrino interactions in the Earth and predicts the spectrum of the $\tau$-lepton and muons that emerge. The $\texttt{nuPyProp}$ outputs are lookup tables of charged lepton exit probabilities and energies that can be used directly or as inputs to the $\texttt{nuSpaceSim}$ code designed to simulate optical and radio signals from extensive air showers induced by the emerging charged leptons. We describe the inputs to the code, demonstrate its flexibility and show selected results for $\tau$-lepton and muon exit probabilities and energy distributions. The $\texttt{nuPyProp}$ code is open source, available on github. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.15590-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.15590) | **Constraints on dark matter annihilation from the FAST observation of the  Coma Berenices dwarf galaxy**  |
|| <mark>Wen-Qing Guo</mark>, et al. -- incl., <mark>Yichao Li</mark> |
|*Appeared on*| *2022-10-03*|
|*Comments*| *9 pages, 4 figures, 1 table, submitted to PRD*|
|**Abstract**| The Galactic center $\gamma$-ray excess, detected by the Fermi-LAT, is a very attractive tentative signal from dark matter annihilation. Searching for associated synchrotron emissions could test the dark matter interpretation for this excess. We point the Five-hundred-meter Aperture Spherical radio Telescope towards Coma Berenices, a dwarf Spheroidal galaxy, for 2-hours of observation, and we find no significant continuum radio emission, which could be attributed to dark matter annihilation, from our target. We set very stringent annihilation cross-section constraints, with roughly an order of magnitude improvement over a large range of masses compared with previous radio searches. The dark matter scenario for the Galactic center $\gamma$-ray excess is in tension with the FAST observation for reasonable choices of astrophysical factors, including diffusion coefficient, diffusion radius and magnetic field. But with the combination of a small diffusion radius and a large diffusion coefficient, the $\mu^{+}\mu^{-}$ channel for the excess could still survive. Further radio observations by the FAST and other radio telescopes may reveal more about the dark matter properties. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.15066-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.15066) | **Global instability by runaway collisions in nuclear stellar clusters:  Numerical tests of a route for massive black hole formation**  |
|| M.C. Vergara, A. Escala, <mark>D.R.G. Schleicher</mark>, B. Reinoso |
|*Appeared on*| *2022-10-03*|
|*Comments*| *11 pages, 6 figures*|
|**Abstract**| The centers of galaxies host nuclear stellar clusters, supermassive black holes, or both, but the origin of this dichotomy is still a mystery. Nuclear stellar clusters are the densest stellar system of the Universe, so they are ideal places for runaway collisions to occur. In these dense clusters it is possible that global instability occurs, triggered by collisions and mergers forming a massive black hole. Here we test a new mechanism to form massive black holes through runaway stellar collisions in nuclear stellar clusters, performing N-body simulations using the code nbody6++gpu. Our idealized models show that there is a critical mass where collisions become very efficient making it possible to form massive black holes in nuclear stellar clusters. The most massive objects reach masses of the order of $10^4-10^5\rm~ M_\odot$. We find that our highest black hole formation efficiency is up to $50\%$ of the stellar mass at the end of the simulation. In real astrophysical systems, the critical mass scale for this transition is expected to occur in stellar clusters of $10^7-10^9\rm~M_\odot$, implying the formation of quite massive central objects. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.15110.md
    + _build/html/tmp_2209.15110/./Fig4New.png
    + _build/html/tmp_2209.15110/./Fig2New.png
    + _build/html/tmp_2209.15110/./HAWCFeb2020.png
exported in  _build/html/2209.15403.md
    + _build/html/tmp_2209.15403/./images/M1/plot_corner.png
    + _build/html/tmp_2209.15403/./images/M1/plot_spectrum.png
    + _build/html/tmp_2209.15403/./images/M2/plot_spectrum.png
    + _build/html/tmp_2209.15403/./images/M3/plot_spectrum.png
    + _build/html/tmp_2209.15403/./images/M4/plot_spectrum.png
    + _build/html/tmp_2209.15403/./images/posteriors/R_pl_Posterior.png
    + _build/html/tmp_2209.15403/./images/posteriors/M_pl_Posterior.png
    + _build/html/tmp_2209.15403/./images/posteriors/P0_Posterior.png
    + _build/html/tmp_2209.15403/./images/posteriors/R_pl_posteriors_legend.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# High Altitude characterization of the Hunga Pressure Wave with Cosmic Rays by the HAWC Observatory

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.15110-b31b1b.svg)](https://arxiv.org/abs/2209.15110)<mark>Appeared on: 2022-10-03</mark> - _Contact about this analysis: A. Sandoval (asandoval@fisica.unam.mx), A. Lara (alara@igeofisica.unam.mx) & H. Le\'on Vargas (hleonvar@fisica.unam.mx)_

</div>
<div id="authors">

<mark><mark>R. Alfaro</mark></mark>, et al. -- incl., <mark><mark>R. Diaz Hernandez</mark></mark>, <mark><mark>S. Hernandez</mark></mark>, <mark><mark>A. Lara</mark></mark>, <mark><mark>J.T. Linnemann</mark></mark>

</div>
<div id="abstract">

**Abstract:** High-energy cosmic rays that hit the Earth can be used to study large-scale atmospheric perturbations. After a first interaction in the upper parts of the atmosphere, cosmic rays produce a shower of particles that sample the atmosphere down to the detector level. The HAWC (High-Altitude Water Cherenkov) cosmic-ray observatory in Central Mexico at 4,100 m elevation detects air shower particles continuously with 300 water Cherenkov detectors with an active area of 12,500 m$^{2}$. On January 15th, 2022, HAWC detected the passage of the pressure wave created by the explosion of the Hunga volcano in the Tonga islands, 9,000 km away, as an anomaly in the measured rate of shower particles. The HAWC measurements are used to characterize the shape of four pressure wave passages, determine the propagation speed of each one, and correlate the variations of the shower particle rates with the barometric pressure changes, extracting a barometric parameter. The profile of the shower particle rate and atmospheric pressure variations for the first transit of the pressure wave at HAWC is compared to the pressure measurements at Tonga island, near the volcanic explosion. This work opens the possibility of using large particle cosmic-ray air shower detectors to trace large atmospheric transient waves. 

</div>

<div id="div_fig1">

<img src="tmp_2209.15110/./Fig4New.png" alt="Fig6" width="100%"/>

**Figure 6. -** Top panel: Variations on the fractional rate of detected shower particles after the effect of the daily oscillation has been subtracted for the time of the first passage of the Hunga pressure wave. The black line shows the data for all photosensors, and the red and blue lines correspond to each tank's two subsets of central and peripheral sensors. The horizontal dashed line marks the fractional rate of the most significant fluctuation before the pressure wave's arrival. The point where the signal crosses this line defines the arrival time of the wave. It is indicated by the location of the arrow. Bottom panel: The black line shows the measured barometric pressure variations, with the arrival of the pressure wave indicated with the vertical arrow. The pressure variations calculated from the HAWC measured rate variations (top panel) using Eq. \ref{eq:deltar} is shown by the red line. \label{fig:Fluctuations} (*fig:Fluctuations*)

</div>
<div id="div_fig2">

<img src="tmp_2209.15110/./Fig2New.png" alt="Fig3" width="100%"/>

**Figure 3. -** Top panel: Traces of the shower particle rates from the HAWC array as a function of time from January 15th, 00:00 UTC to January 18th, 06:00 UTC. Shown in black are the rates for all considered photosensors; in blue, the subset of peripheral detectors of each tank is scaled up by a 1.5 factor; in red, the subset of central detectors is scaled up by a 2.9 factor. The scale factors were used to display all the data with similar rate values. One sees the bi-diurnal oscillations and superimposed on them the signature of 4 passes of the Hunga explosion pressure wave delimited by the vertical lines in regions A-D. The spike after the D pass is due to an electric storm at the site. Bottom panel: Pressure variations from January 15th, 2022, 0:00 UTC to January 18th, 2022, 6:00 UTC. One sees the twice-daily oscillations are anti-correlated to the particle rate, and superimposed on them the pressure changes due to the passage of the pressure wave (A-D). The short path wave passages are (A) and (C), and the long path passages are (B) and (D).\label{fig:Traces} (*fig:Traces*)

</div>
<div id="div_fig3">

<img src="tmp_2209.15110/./HAWCFeb2020.png" alt="Fig1" width="100%"/>

**Figure 1. -** The HAWC gamma- and cosmic-ray observatory on the slopes of the Sierra Negra Volcano, Puebla, Mexico, at 4,100 m elevation. Air shower particles are detected in the 300 water tanks by the Cherenkov light they emit when passing through the water. The picture also shows the detector upgrade that consists of smaller WCDs that surround the main array.\label{fig:HAWCObs} (*fig:HAWCObs*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand{\baselinestretch}{1.0}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand{$\baselinestretch$}{1.0}$</div>



<div id="title">

# Atmospheric retrievals for LIFE and other future space missions: the importance of mitigating systematic effects

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.15403-b31b1b.svg)](https://arxiv.org/abs/2209.15403)<mark>Appeared on: 2022-10-03</mark> - _24 pages, 12 figures. Proceedings SPIE Volume 12180, Space Telescopes and Instrumentation 2022: Optical, Infrared, and Millimeter Wave; 121803L (2022)_

</div>
<div id="authors">

Eleonora Alei, et al.

</div>
<div id="abstract">

**Abstract:** Atmospheric retrieval studies are essential to determine the science requirements for future generation missions, such as the Large Interferometer for Exoplanets (LIFE). The use of heterogeneous absorption cross-sections might be the cause of systematic effects in retrievals, which could bias a correct characterization of the atmosphere.  In this contribution we quantified the impact of differences in line list provenance, broadening coefficients, and line wing cut-offs in the retrieval of an Earth twin exoplanet orbiting a Sun-like star at 10 pc from the observer, as it would be observed with LIFE. We ran four different retrievals on the same input spectrum, by varying the opacity tables that the Bayesian retrieval framework was allowed to use. We found that the systematics introduced by the opacity tables could bias the correct estimation of the atmospheric pressure at the surface level, as well as an accurate retrieval of the abundance of some species in the atmosphere (such as\ce{CO2}and\ce{N2O}). We argue that differences in the line wing cut-off might be the major source of errors. We highlight the need for more laboratory and modeling efforts, as well as inter-model comparisons of the main radiative transfer models and Bayesian retrieval frameworks. This is especially relevant in the context of LIFE and future generation missions, to identify issues and critical points for the community to jointly work together to prepare for the analysis of the upcoming observations.

</div>

<div id="div_fig1">

<img src="tmp_2209.15403/./images/M1/plot_corner.png" alt="Fig6" width="100%"/>

**Figure 6. -** example\label{fig:corner_1} Corner plot for the posterior distributions from the retrievals of Run 1 (color-coded according to Table \ref{tab:setups}). The black lines indicate the expected values for every parameter. On the diagonal, we show the marginalized posteriors for each parameter, with median and 1-$\sigma$ uncertainties as dashed gray lines.
 (*fig:corner_1*)

</div>
<div id="div_fig2">

<img src="tmp_2209.15403/./images/M1/plot_spectrum.png" alt="Fig1.1" width="25%"/><img src="tmp_2209.15403/./images/M2/plot_spectrum.png" alt="Fig1.2" width="25%"/><img src="tmp_2209.15403/./images/M3/plot_spectrum.png" alt="Fig1.3" width="25%"/><img src="tmp_2209.15403/./images/M4/plot_spectrum.png" alt="Fig1.4" width="25%"/>

**Figure 1. -** Spectra\label{fig:spectra}
Retrieved spectra compared to the input spectrum (black line) for the various runs. The gray error bars indicate the LIFE$\textsc${sim} uncertainty. The color-shaded areas represent the confidence envelopes
(darker shading corresponds to higher confidence). (*fig:spectra*)

</div>
<div id="div_fig3">

<img src="tmp_2209.15403/./images/posteriors/R_pl_Posterior.png" alt="Fig4.1" width="25%"/><img src="tmp_2209.15403/./images/posteriors/M_pl_Posterior.png" alt="Fig4.2" width="25%"/><img src="tmp_2209.15403/./images/posteriors/P0_Posterior.png" alt="Fig4.3" width="25%"/><img src="tmp_2209.15403/./images/posteriors/R_pl_posteriors_legend.png" alt="Fig4.4" width="25%"/>

**Figure 4. -** Posteriors\label{fig:posteriors_bulk}
 Posterior density distributions for the retrieved bulk parameters (planet radius, planet mass, ground pressure) for the four runs. The vertical, dashed lines mark the expected values for each parameter. For $R_{pl}$ and $\mathrm{log_{10}(M_{pl})}$, dotted lines show the corresponding priors for comparison. (*fig:posteriors_bulk*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

52  publications in the last 7 days.
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers